In [1]:
from ovg_experiments.simulated_data import (
    generate_simulated_data, DataGenSettings, 
    ExperimentType, SimulatedData
)
from os.path import join
from datetime import datetime
from pathlib import Path
from typing import Dict

from ovg.predictors import (
    MarginalPredictor, ProposedPredictor, 
    OptimalPredictor, ImputedPredictor,
    Predictor, PredictorType
)
from ovg.evaluation import compute_zero_shot_loss, visualize_zero_shot
import numpy as np
import torch
import random

In [2]:
# Configuration

experiment_type = ExperimentType.polynomial

results_dir = (                                                                                                                                                                                                               
        Path.cwd() / f'zero_shot_learning_polynomial_{datetime.now().strftime("%y_%m_%d_%H_%M_%S")}'                                                                                                                                           
)                                                                      
results_dir.mkdir(parents=True)

datagen_settings = DataGenSettings(
        num_samples=10000,                                                                                                                                                                                                        
        split_fraction=0.9,                                                                                                                                                                                                       
        noise_var=0.1,                                                                                                                                                                                                            
        noise_skew=0.0,                                                                                                                                                                                                           
        noise_mean=0.0
) 

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
# Generate Dataset

dataset: SimulatedData = generate_simulated_data(experiment_type, datagen_settings)

In [4]:
# fitting predictors

predictors: Dict[PredictorType,Predictor] = {
        PredictorType.oracle: OptimalPredictor(),
        PredictorType.proposed: ProposedPredictor(),
        PredictorType.marginal: MarginalPredictor(),
        PredictorType.imputation: ImputedPredictor()
}

for predictor in predictors.values():
    predictor.fit(dataset.data_source, dataset.data_target)

In [5]:
# compute zero-shot loss

losses: Dict[PredictorType, float] = compute_zero_shot_loss(
    predictors[PredictorType.proposed], 
    predictors, 
    dataset.data_target, 
    num_samples=1000
)
for predictor_type, loss in losses.items():
    print(f"{predictor_type.name}\t\t{loss:.3f}")

oracle		0.158
proposed		0.000
marginal		3.133
imputation		2.067


In [8]:
# plotting

visualize_zero_shot(
    predictors, dataset.data_target, save_dir=results_dir, num_samples=1000,
    display=True, figsize = (48,48), dpi=200, fontsize=30
)

print(f"plots saved in {results_dir}")

plots saved in /home/vberenz/Workspaces/ovg/Out-of-Variable-Generalization/experiments/zero_shot_learning_polynomial_24_03_19_15_20_26
